#### <center> The prupose of this notebook is to plot </center>
# <center> Heatmaps 🌈🌈🌈</center>

In [ ]:
%reset -sf

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
# Reading data

from pandas import read_csv

train = read_csv('/kaggle/input/tabular-playground-series-mar-2022/train.csv')
test = read_csv('/kaggle/input/tabular-playground-series-mar-2022/test.csv')
sample_subm = read_csv('/kaggle/input/tabular-playground-series-mar-2022/sample_submission.csv')

In [ ]:
# Dates format

from pandas import to_datetime

train['time'] = to_datetime(train['time'])

In [ ]:
# Encode direction column

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['direction'])

train['direction_c'] = le.transform(train['direction'])

In [ ]:
# Instructions says 65 different roads

mask = (train['x']==0) & (train['y']==0) & (train['direction_c']==0)
train[mask]

In [ ]:
# Inspecting Roads

train['x'].unique()
train['y'].unique()
train['direction_c'].unique()

from pandas import crosstab

crosstab(train['x'], train['direction_c'], values='congestion', aggfunc='count')
crosstab(train['y'], train['direction_c'], values='congestion', aggfunc='count')

# There are some combinations not present, therefore 
# total number of roads is not 3 * 4 * 8 = 96, rather the aforementioned 65

## Now, on to visualizing some Heatmaps 🌈🌈🌈

In [ ]:
# Inspecting congestion

import matplotlib.pyplot as plt
from pandas import DataFrame
from numpy import arange, array

roads = train.set_index('time')

pv = roads.groupby(['x', 'y', 'direction_c']).resample('20T')[['congestion']].mean()
pv = pv.unstack(level=[0,1,2]).droplevel(axis=1, level=0).sort_index(ascending=False)
#pv.index

each = 500
ticks = [(t, tl) for t, tl, in zip(arange(len(pv)), pv.index) if t % each == 0]

fig, ax = plt.subplots(figsize=(15,10))
im = ax.pcolor(pv, cmap=plt.cm.RdYlGn_r, linewidths=.1)
_ = ax.set_yticks(array([t[0] for t in ticks]) + 0.5, minor=False)
_ = ax.set_yticklabels([t[1].to_period('D') for t in ticks])
#_ = ax.set_xticks(range(nba_sort.shape[1]) + 0.5, minor=False)
_ = ax.set_xlabel('Roads')
_ = ax.set_ylabel('Date', rotation=360, labelpad=20)
_ = ax.xaxis.tick_top()
_ = ax.xaxis.set_label_position('top') 
_ = ax.set_title('Heatmap: Roads Congestion by Date')

cb = fig.colorbar(im, drawedges=False)
_ = cb.set_label('Congestion\nLevel', rotation=360, labelpad=25)

# It seems that in general there are roads that are more congested than others

## Previous heatmap was with raw data (data each 20 minutes), let's do it summarized it each day

In [ ]:
# Each day

pv = roads.groupby(['x', 'y', 'direction_c']).resample('24H')[['congestion']].mean()
pv = pv.unstack(level=[0,1,2]).droplevel(axis=1, level=0).sort_index(ascending=False)
#pv.index

each = 10
ticks = [(t, tl) for t, tl, in zip(arange(len(pv)), pv.index) if t % each == 0]

fig, ax = plt.subplots(figsize=(15,10))
im = ax.pcolor(pv, cmap=plt.cm.RdYlGn_r, linewidths=.1)
_ = ax.set_yticks(array([t[0] for t in ticks]) + 0.5, minor=False)
_ = ax.set_yticklabels([t[1].to_period('D') for t in ticks])
#_ = ax.set_xticks(range(nba_sort.shape[1]) + 0.5, minor=False)
_ = ax.set_xlabel('Roads')
_ = ax.set_ylabel('Date', rotation=360, labelpad=20)
_ = ax.xaxis.tick_top()
_ = ax.xaxis.set_label_position('top') 
_ = ax.set_title('Heatmap: Roads Congestion by Date')

cb = fig.colorbar(im, drawedges=False)
_ = cb.set_label('Congestion\nLevel', rotation=360, labelpad=25)

## We start to see some nice patterns:
### - There are roads more generally congested than others,
### - Many roads seems to have weekly patterns,
### - Also, it seems congestion decrease at weekends (judging by the brightening of small patches),
### - 64th road (the mostly green-congested) does not seem to have a weekly pattern, rather random,
### - Lastly, there seems to be some dates that roads has generally less congestion, i.e. ~1991-4-30

## Now, let's try each week

In [ ]:
# Each week

pv = roads.groupby(['x', 'y', 'direction_c']).resample('7D')[['congestion']].mean()
pv = pv.unstack(level=[0,1,2]).droplevel(axis=1, level=0).sort_index(ascending=False)
#pv.index

each = 2
ticks = [(t, tl) for t, tl, in zip(arange(len(pv)), pv.index) if t % each == 0]

fig, ax = plt.subplots(figsize=(15,10))
im = ax.pcolor(pv, cmap=plt.cm.RdYlGn_r, linewidths=.1)
_ = ax.set_yticks(array([t[0] for t in ticks]) + 0.5, minor=False)
_ = ax.set_yticklabels([t[1].to_period('D') for t in ticks])
#_ = ax.set_xticks(range(nba_sort.shape[1]) + 0.5, minor=False)
_ = ax.set_xlabel('Roads')
_ = ax.set_ylabel('Date', rotation=360, labelpad=20)
_ = ax.xaxis.tick_top()
_ = ax.xaxis.set_label_position('top') 
_ = ax.set_title('Heatmap: Roads Congestion by Date')

cb = fig.colorbar(im, drawedges=False)
_ = cb.set_label('Congestion\nLevel', rotation=360, labelpad=25)

### Main findings here are:
### - Last week, there seems to be a general decrease in congestion (algorithms beware)
### - Can't say there are weekly patterns...

### How about each month...

# Hope it's useful to you!

### Bye!